In [2]:
import requests
import pandas as pd
import pycountry


In [3]:
# Ülkelerin iso3 kodlarını alıyorum

def get_iso3(country_name):
    try:
        return pycountry.countries.lookup(country_name).alpha_3
    except LookupError:
        return None

countries = [
"Afghanistan","Albania","Algeria","American Samoa","Angola","Anguilla (United Kingdom)","Antigua and Barbuda",
"Argentina","Armenia","Aruba","Australia","Austria","Azerbaijan","Bahamas","Bangladesh","Barbados","Belarus",
"Belgium","Belize","Benin","Bhutan","Bolivia, Plurinational State…","Bonaire, Saint Eustatius and…","Bosnia and Herzegovina",
"Botswana","Brazil","Brunei Darussalam","Bulgaria","Burkina Faso","Burundi","Cambodia","Cameroon","Canada","Cape Verde",
"Cayman Islands","Central African Republic","Chad","Chile","China","Colombia","Comoros","Congo","Congo, The Democratic…",
"Cook Islands","Costa Rica","Croatia","Cuba","Curaçao","Cyprus","Czech Republic","Côte d'Ivoire","Denmark","Djibouti",
"Dominica","Dominican Republic","Ecuador","Egypt","El Salvador","Equatorial Guinea","Eritrea","Estonia","Eswatini",
"Ethiopia","Falkland Islands (Malvinas)","Fiji","France","French Polynesia","Gabon","Gambia","Georgia","Germany",
"Ghana","Global","Greece","Grenada","Guadeloupe (France)","Guam","Guatemala","Guinea","Guinea-Bissau","Guyana",
"Haiti","Vatican","Honduras","Hungary","Iceland","India","Indonesia","Iran","Iraq","Ireland","Israel","Italy","Jamaica",
"Japan","Jordan","Kazakhstan","Kenya","Kiribati","Korea, Democratic People's…","Korea, Republic of","Kuwait",
"Kyrgyzstan","Lao People's Democratic…","Latvia","Lebanon","Lesotho","Liberia","Libya","Lithuania","Luxembourg","Madagascar",
"Malawi","Malaysia","Maldives","Mali","Malta","Marshall Islands","Martinique (France)","Mauritania","Mauritius",
"Mexico","Micronesia, Federated States…","Moldova, Republic of","Mongolia","Montenegro","Montserrat","Morocco",
"Mozambique","Myanmar","Namibia","Nauru","Nepal","Netherlands","New Caledonia","New Zealand","Nicaragua","Niger",
"Nigeria","Niue","North Macedonia","Norway","occupied Palestinian territory","Oman","Pakistan","Palau","Panama",
"Papua New Guinea","Paraguay","Peru","Philippines","Poland","Portugal","Puerto Rico (United States)","Qatar",
"Romania","Russian Federation","Rwanda","Saint Barthélemy (France)","Saint Helena, Ascension and…","Saint Kitts and Nevis",
"Saint Lucia","Saint Martin (France)","Saint Vincent and the…","Samoa","San Marino","Saudi Arabia","Senegal",
"Serbia","Serbia and Montenegro ","Seychelles","Sierra Leone","Singapore","Sint Maarten (Dutch part)","Slovakia",
"Slovenia","Solomon Islands","Somalia","South Africa","South Sudan","Spain","Sri Lanka","Sudan","Suriname",
"Sweden","Switzerland","Syrian Arab Republic","São Tomé and Príncipe","Taiwan, Province of China","Tajikistan",
"Tanzania","Thailand","Timor-Leste","Togo","Tokelau","Tonga","Trinidad and Tobago","Tunisia","Turkmenistan","Turks and Caicos Islands",
"Tuvalu","Türkiye","Uganda","Ukraine","United Arab Emirates","United Kingdom","United States","Uruguay","Uzbekistan",
"Vanuatu","Venezuela, Bolivarian…","Viet Nam","Virgin Islands, British","Western Sahara","Yemen","Zambia","Zimbabwe"
]

iso3_codes = {country: get_iso3(country) for country in countries}
print(iso3_codes)

# Buradan sonra None olanlar elle txt dosyasından düzeltildi

{'Afghanistan': 'AFG', 'Albania': 'ALB', 'Algeria': 'DZA', 'American Samoa': 'ASM', 'Angola': 'AGO', 'Anguilla (United Kingdom)': None, 'Antigua and Barbuda': 'ATG', 'Argentina': 'ARG', 'Armenia': 'ARM', 'Aruba': 'ABW', 'Australia': 'AUS', 'Austria': 'AUT', 'Azerbaijan': 'AZE', 'Bahamas': 'BHS', 'Bangladesh': 'BGD', 'Barbados': 'BRB', 'Belarus': 'BLR', 'Belgium': 'BEL', 'Belize': 'BLZ', 'Benin': 'BEN', 'Bhutan': 'BTN', 'Bolivia, Plurinational State…': None, 'Bonaire, Saint Eustatius and…': None, 'Bosnia and Herzegovina': 'BIH', 'Botswana': 'BWA', 'Brazil': 'BRA', 'Brunei Darussalam': 'BRN', 'Bulgaria': 'BGR', 'Burkina Faso': 'BFA', 'Burundi': 'BDI', 'Cambodia': 'KHM', 'Cameroon': 'CMR', 'Canada': 'CAN', 'Cape Verde': None, 'Cayman Islands': 'CYM', 'Central African Republic': 'CAF', 'Chad': 'TCD', 'Chile': 'CHL', 'China': 'CHN', 'Colombia': 'COL', 'Comoros': 'COM', 'Congo': 'COG', 'Congo, The Democratic…': None, 'Cook Islands': 'COK', 'Costa Rica': 'CRI', 'Croatia': 'HRV', 'Cuba': 'CUB'

In [4]:
# ülke-iso3 kodu txt dosyasını dict'e aktarıyorum
dict_from_file = {}
with open('country_codes.txt', 'r') as file:
    for line in file:
        key, value = line.strip().split(':')
        dict_from_file[key] = value


In [7]:
# Arada bir sıkıntı çıktı excel'in tanımadığı karakter ile ilgili onu çözüyor. tşk openai
def remove_non_printable_chars(df):
    # Remove non-printable characters from all string columns
    for col in df.select_dtypes(include=['object']):
        df[col] = df[col].apply(lambda x: ''.join(filter(str.isprintable, x)) if isinstance(x, str) else x)
    return df

In [8]:
# Ana api call fonksiyonu
# Paginationdan nefret ediyorum.

def query_aid_flows_to_df(country_iso3, plan_id=None, group_by=None, filter_by=None, year=None, format='json'):
    base_url = "https://api.hpc.tools/v1/public/fts/flow"
    params = {
        'countryISO3': country_iso3,
        'format': format
    }
    if plan_id:
        params['planId'] = plan_id
    if group_by:
        params['groupby'] = group_by
    if filter_by:
        params['filterBy'] = filter_by
    if year:
        params['year'] = year  
    
    all_flows = []  # List to store all flows across pages
    page = 1  # Starting page
    while True:
        params['page'] = page
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            data = response.json()['data']['flows']
            if not data:  # Break the loop if no data is returned
                break
            all_flows.extend(data)
            page += 1  # Move to the next page
        else:
            print(f"Error: {response.status_code}")
            break

    df = pd.DataFrame(all_flows)
    return df




In [11]:
"""
query_aid_flows_to_df fonksiyonunu anlamak için bu satırı çalıştırabilirsiniz
1150 flash appeal'ın kodu
"""

df = query_aid_flows_to_df('TUR',plan_id=1150, year=2023)
df.tail()

,id,amountUSD,budgetYear,childFlowIds,contributionType,createdAt,date,decisionDate,description,grandBargainEarmarkingType,...,parentFlowId,status,updatedAt,versionId,sourceObjects,destinationObjects,boundary,onBoundary,reportDetails,refCode
457,305223,94367,None,None,financial,2024-04-05T02:25:30.237Z,2024-04-04T00:00:00Z,2024-01-01T00:00:00Z,OCHA Carry over from 2023 to 2024,[softly earmarked],...,NaN,paid,2024-04-05T10:35:21.144Z,1,"[{'type': 'Plan', 'id': '1150', 'name': 'Türki...","[{'type': 'Organization', 'id': '4396', 'name'...",outgoing,single,"[{'sourceType': 'Primary', 'organization': 'Of...",2660:2024:OCT
458,305575,1233319,2022,None,financial,2024-04-15T15:03:03.513Z,2023-11-06T00:00:00Z,2023-11-06T00:00:00Z,Protection and assistance to people in need of...,[earmarked],...,294255.0,commitment,2024-04-15T15:03:03.513Z,1,"[{'type': 'Organization', 'id': '3774', 'name'...","[{'type': 'Plan', 'id': '1150', 'name': 'Türki...",incoming,single,"[{'sourceType': 'Primary', 'organization': 'Un...",NV 1991 - 23-GOV-IT-011-01
459,305576,887572,2022,None,financial,2024-04-15T15:04:05.842Z,2023-11-06T00:00:00Z,2023-11-06T00:00:00Z,Protection and assistance to people in need of...,[earmarked],...,294255.0,commitment,2024-04-15T15:04:05.842Z,1,"[{'type': 'Organization', 'id': '3774', 'name'...","[{'type': 'Plan', 'id': '1150', 'name': 'Türki...",incoming,single,"[{'sourceType': 'Primary', 'organization': 'Un...",NV 1991 - 23-GOV-IT-011-01
460,305944,2177,None,None,financial,2024-04-17T15:12:30.600Z,2023-01-01T00:00:00Z,2023-01-01T00:00:00Z,Food Security & Livelihoods,[earmarked],...,NaN,paid,2024-04-17T15:12:30.600Z,1,"[{'type': 'Organization', 'id': '4543', 'name'...","[{'type': 'Plan', 'id': '1150', 'name': 'Türki...",incoming,single,"[{'sourceType': 'Primary', 'organization': 'Wo...",10047987\tALLOCATION FILE\tWPD-C-06298-06\tTR03
461,305945,21160,None,None,financial,2024-04-17T15:23:12.158Z,2023-11-21T00:00:00Z,2023-11-21T00:00:00Z,Food Security & Livelihoods,[earmarked],...,NaN,paid,2024-04-17T15:23:12.158Z,1,"[{'type': 'Organization', 'id': '4844', 'name'...","[{'type': 'Plan', 'id': '1150', 'name': 'Türki...",incoming,single,"[{'sourceType': 'Primary', 'organization': 'Wo...",10047437\tREPROGRAMMING\tHUN-C-00036-01\tTR03


In [ ]:
# !!!! Bu satırı çalıştırırsanız benim drive'a yüklediğim dosyaları oluşturmaya başlar ve uzun sürer.
#iso3 None değilse datayı gönder gelsin. 
# aradaki printler testing için var. Yoksayın. 
# country_iso3,plan_id ve year parametreleri ile filtreleyebiliyoruz. 
# Örneğin query_aid_flows_to_df('TUR',plan_id=1150, year=2023)

for k, v in dict_from_file.items():
    print(1)
    if v != 'None':
        country_iso3 = v  # Country ISO code
        print(2)
        plan_id = None
        print(3)
        year = None  # Tümünü alalım
        print(4)
        df = query_aid_flows_to_df(country_iso3, plan_id=plan_id, year=year)
        df = remove_non_printable_chars(df)  # Clean the DataFrame
        print(5)
        print(v, '\n')
        try:
            print(df['flowType'].iloc[-1])
        except KeyError as e:
            print('No data for the specified parameters/country')
        except IndexError as e:
            print('DataFrame is empty, no data to show.')
    else:
        continue
    df.to_excel(f'./{k}_{v}.xlsx')
